In [4]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pymysql
import pandas as pd
import warnings
import os
pymysql.install_as_MySQLdb()
warnings.filterwarnings("ignore")

### Make sure only disease.csv files are showing

In [2]:
filelist = os.listdir("Raw_Data")
filelist

['Pertussis.csv',
 'Typhoid.csv',
 'Hep A.csv',
 'Hep B.csv',
 'Smallpox.csv',
 'Varicella.csv',
 'Diptheria.csv',
 'Rubella.csv',
 'Measles.csv',
 'Tuberculosis.csv',
 'Mumps.csv']

### Pulls in csv file and formats dataframes to be appended into one clean dataframe

In [3]:
cleaned_df = pd.DataFrame({'Disease': [], 'Fatalities': [], 'State': [], 'City': [], 'PeriodStartDate': [], 'PeriodEndDate': []})
for file in filelist:
    df = pd.read_csv("Raw_Data/"+file)
    df_new = df[['ConditionName', 'Fatalities', 'Admin1Name', 'CityName', 'PeriodStartDate', 'PeriodEndDate']]
    df_renamed = df_new.rename(columns={'ConditionName':'Disease','Admin1Name':'State','CityName':'City'})
    df_renamed['PeriodStartDate'] = df_renamed['PeriodStartDate'].apply(pd.to_datetime, errors='coerce')
    df_renamed['PeriodEndDate'] = df_renamed['PeriodEndDate'].apply(pd.to_datetime, errors='coerce')
    cleaned_df = cleaned_df.append(df_renamed, ignore_index=True)   

### Save cleaned_df as .csv

In [ ]:
cleaned_df.to_csv("Output/cleaned_df.csv", index=False, header=True)

### Create sqlite database connection

In [5]:
Base = declarative_base()
class Disease(Base):
    __tablename__ = 'diseases'
    id = Column(Integer, primary_key=True)
    Disease = Column(String(255))
    Fatalities = Column(Integer)
    State = Column(String(255))
    City = Column(String(255))
    PeriodStartDate = Column(String(255))
    PeriodEndDate = Column(String(255))

In [6]:
engine = create_engine("sqlite:///diseases.sqlite")
conn = engine.connect()
Base.metadata.create_all(conn)
session = Session(bind=engine)

In [7]:
cleaned_df.to_sql("diseases", conn, if_exists='append', index=False)